# Segmenting and Clustering Neighborhoods in Toronto by the following criteria:
### Question 1: Web scrape data from Wikipedia and create custom data frame (borough & neighborhood)
### Question 2: Get the latitude and the longitude coordinates of each neighborhood and add them to dataframe
### Question 3: Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto

#### Installing BeautifulSoup to scrape data neighborhood data from Toronto (Wikipedia) along with pandas and numpy (Note: Data is from 2 January, 2020)

In [4]:
import requests
URL = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=935851093').text

In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(URL,'lxml')
# print(soup.prettify())

#### Tabular data can be found by looking for classname = 'wikitable sortable', and hence that is the only data to extract

In [6]:
table = soup.find('table',{'class':'wikitable sortable'})
# table

In [7]:
print(table.tr.text)


Postcode
Borough
Neighborhood



In [8]:
headers="Postcode,Borough,Neighbourhood"

#### Getting all values from tuples (tr) and separating data (td) with delimiter ','

In [9]:
table1 = ""
for tr in table.find_all('tr'):
    row1 = ""
    for tds in tr.find_all('td'):
        row1 = row1 + "," + tds.text
    table1 = table1 + row1[1:]
# print(table1)

#### Writing data into .csv format for convenience, and then convert to dataframe and perform formatting operations

In [10]:
file = open("toronto.csv","wb")
file.write(bytes(table1,encoding="ascii",errors="ignore"))

8708

In [11]:
import pandas as pd
df = pd.read_csv('toronto.csv',header=None)
df.columns = ["Postalcode","Borough","Neighbourhood"]

In [12]:
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Find tuples with Borough = 'Not Assigned', and then drop them

In [13]:
# Get names of indexes for which column Borough has value "Not assigned"
indexNames = df[ df['Borough'] =='Not assigned'].index

# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)

In [14]:
df.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [15]:
df.loc[df['Neighbourhood'] =='Not assigned' , 'Neighbourhood'] = df['Borough']
df.head(10)

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [16]:
result = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)
df_new=result.reset_index()

### Answer to Question 1 with .shape requirement

## df_new.head(15)

In [17]:
df_new.shape

(103, 3)

## Question 2
### importing Geocoder package to create df with latitude and longitude columns by reading csv file http://cocl.us/Geospatial_data

In [18]:
!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data
import pandas as pd
df2 = pd.read_csv('Toronto_long_lat_data.csv')
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
df2.columns = ['Postalcode', 'Latitude', 'Longitude']

In [22]:
df2.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
df_toronto = pd.merge(df_new, df2[['Postalcode','Latitude','Longitude']],on='Postalcode')
df_toronto.head(10)

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


## Question 3
### Cluster the Toronto neighborhoods using geopy library

In [25]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Done')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.0               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                       

In [26]:

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [28]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto